In [1]:
from sklearn import tree
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re


In [2]:
import spacy
from spacy.lang.en import English
import contextualSpellCheck

In [38]:
train_data = pd.read_csv('nlp-getting-started/train.csv', index_col=0)
test_data = pd.read_csv('nlp-getting-started/test.csv', index_col=0)

In [39]:
train_data.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [40]:
print(train_data.shape, test_data.shape)

(7613, 4) (3263, 3)


In [41]:
train_data.isnull().sum()

keyword       61
location    2533
text           0
target         0
dtype: int64

## Dealing with mislabeling

In [42]:
mislabeled_data = train_data.groupby(['text']).nunique().sort_values(by='target', ascending=False)
mislabeled_data = mislabeled_data[mislabeled_data['target'] > 1]['target']
mislabeled_data.index.tolist()

['like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit',
 'Hellfire! We don\x89Ûªt even want to think about it or mention it so let\x89Ûªs not do anything that leads to it #islam!',
 "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'",
 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!',
 'To fight bioterrorism sir.',
 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE',
 '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption',
 '#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect',
 'He came to a land which was engulfed in tribal war and turned it into a land 

In [79]:
def fix_target(l):
    true_labels = [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
    for i in range(len(l)):
        indexes = train_data.loc[train_data['text'] == l[i]].index
        for index in indexes:
            train_data.loc[index, 'target'] = true_labels[i]           

In [81]:
l = mislabeled_data.index.tolist()
fix_target(l)

## Separate Text

In [7]:
text = pd.concat([train_data.text, test_data.text])

In [8]:
text.head()

id
1    Our Deeds are the Reason of this #earthquake M...
4               Forest fire near La Ronge Sask. Canada
5    All residents asked to 'shelter in place' are ...
6    13,000 people receive #wildfires evacuation or...
7    Just got sent this photo from Ruby #Alaska as ...
Name: text, dtype: object

In [9]:
nlp = English()  # Includes English data
nlp = spacy.load("en_core_web_sm")
lemmatizer = nlp.get_pipe("lemmatizer")


In [10]:
def text_to_token(text):
    return nlp(text)
#     token_list = []
#     text = nlp(text)
#     for token in text:
#         token_list.append(token)#.text)
#     return token_list    

In [11]:
# tmp = text_to_token(text)
text = text.apply(text_to_token)

### Remove stopwords

In [12]:
stopwords = nlp.Defaults.stop_words

In [13]:
def remove_stopwords(text):
    return [word for word in text if word not in stopwords] 

In [14]:
text = text.apply(remove_stopwords)

### Remove short words

In [15]:
def remove_short_words(text):
    return [word for word in text if len(word) > 3]

In [16]:
text = text.apply(remove_short_words)

### Remove punctuation

In [17]:
from string import punctuation

In [18]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
def remove_punctuation(text):
    return [word for word in text if word.text not in punctuation and word.text not in ['\n']] 

In [20]:
text = text.apply(remove_punctuation)

### Remove @username

In [21]:
def remove_username(txt):
    pattern = '@[/w]*'
    return [word for word in txt if not re.match(pattern, word.text)] 

In [22]:
text = text.apply(remove_username)

### Remove links

In [23]:
def remove_links(text):
    pattern = "(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)"
    return [word for word in text if not re.match(pattern, word.text)] 

In [24]:
remove_links(text[93])

[huge, fire, Wholesale, markets, ablaze]

In [25]:
text = text.apply(remove_links)

### Spelling correction


In [24]:
from autocorrect import Speller

In [25]:
spell = Speller()

In [117]:
# import contextualSpellCheck

In [134]:
# import spacy
# nlp = spacy.load('en_core_web_sm')
# contextualSpellCheck.add_to_pipe(nlp)
# doc = nlp('Income was $9.4 milion compared to the prior year of $2.7 milion.')

# print(doc._.performed_spellCheck) #Should be True
# print(doc._.outcome_spellCheck) #Income was $9.4 million compared to the prior year of $2.7 million.

True
Income was $9.4 million compared to the prior year of $2.7 million.


In [121]:
# nlp.add_pipe('sentencizer')

In [245]:
def correct_spelling(text):
    return [spell(word) for word in text]

In [249]:
text[117:118].apply(correct_spelling)

id
170    [I, seeing, Issues, aftershock]
Name: text, dtype: object

In [251]:
for i in range(len(text)):
#     text[i] = text[i].apply(correct_spelling)
    text[i] = correct_spelling(text[i])
    if i % 10 == 0:
        print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [273]:
text_with_spelling = text.copy()

In [272]:
text.head()

id
1    [Our, Deeds, Reason, earthquake, May, ALL, For...
4        [Forest, fire, near, La, Range, Task, Canada]
5    [All, residents, asked, shelter, place, notifi...
6    [13,000, people, receive, wildfires, evacuatio...
7    [Just, got, sent, photo, Ruby, Alaska, smoke, ...
Name: text, dtype: object

In [255]:
text = text.apply(remove_stopwords)

In [70]:
text.head()

id
1           [deed, Reason, earthquake, ALLAH, forgive]
4                   forest fire near Ronge Sask Canada
5    resident shelter place notify officer evacuati...
6    [13,000, people, receive, wildfire, evacuation...
7    [send, photo, Ruby, Alaska, smoke, wildfires, ...
Name: text, dtype: object

## Dealing with mislabeling

In [37]:
mislabeled_data = train_data.groupby(['text']).nunique().sort_values(by='target', ascending=False)
mislabeled_data = mislabeled_data[mislabeled_data['target'] > 1]['target']
mislabeled_data.index.tolist()

['like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit',
 'Hellfire! We don\x89Ûªt even want to think about it or mention it so let\x89Ûªs not do anything that leads to it #islam!',
 "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'",
 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!',
 'To fight bioterrorism sir.',
 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE',
 '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption',
 '#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect',
 'He came to a land which was engulfed in tribal war and turned it into a land 

In [34]:
train_data.loc[train_data['text'] == l[0]].target

id
1760    1
1950    0
Name: target, dtype: int64

In [35]:
fixed = (mislabeled_data.index.tolist())

/Users/yulialysenko/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [36]:
df_mislabeled

NameError: name 'df_mislabeled' is not defined

In [86]:
train_data[train_data['text'] == "like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit"]

,keyword,location,text,target
id,,,,
1760,buildings%20burning,dallas,like for the music video I want some real acti...,1
1950,burning%20buildings,dallas,like for the music video I want some real acti...,0


In [78]:
train_data.target.unique()

array([1, 0])

In [75]:
train_data[train_data['target'] > 1]

,keyword,location,text,target
id,,,,


### Lemmatization

In [363]:

# nlp = spacy.load("en_core_web_sm")


In [39]:
l = text[1]
# l = train_data.text
# doc = nlp()# running runner")
print([token.lemma_ for token in l])

AttributeError: 'str' object has no attribute 'lemma_'

In [27]:
def lemmatize(text):
    return[token.lemma_ for token in text]

In [28]:
text = text.apply(lemmatize)

## From lemma back to token

In [40]:
text

id
1         [deed, Reason, this, earthquake, ALLAH, forgive]
4                [forest, fire, near, Ronge, Sask, Canada]
5        [resident, shelter, place, notify, officer, ot...
6        [13,000, people, receive, wildfire, evacuation...
7        [just, send, this, photo, from, Ruby, Alaska, ...
                               ...                        
10861    [earthquake, SAFETY, ANGELES, SAFETY, fastener...
10865    [storm, than, last, hurricane, city&amp;3other...
10868                   [Green, Line, derailment, Chicago]
10874                 [issue, Hazardous, Weather, Outlook]
10875    [CityofCalgary, have, activate, Municipal, Eme...
Name: text, Length: 10876, dtype: object

In [41]:
text = text.apply(remove_stopwords)

In [43]:
def list_to_tokens(text):
    return [nlp(word) for word in text]
#     token_list = []
# #     text = nlp(text)
#     for token in text:
#         token_list.append(nlp(token)#.text)
#     return token_list  

In [44]:
token_text = text.apply(list_to_tokens)

## Vectorizing

In [505]:
def vectorize(line):
    return [word.vector for word in line]
    

In [525]:
token_text[0:5].apply(vectorize)

id
1    [[-0.027740717, -0.11559065, 0.14615542, -0.11...
4    [[3.0948722, 0.90669054, 0.21063933, -0.480993...
5    [[-1.2147622, 0.49894887, 0.28072032, -0.30333...
6    [[0.5988686, 2.0787547, 0.22975582, -0.5023366...
7    [[-0.5375247, 1.4861925, -0.7751549, 0.3462769...
Name: text, dtype: object

In [526]:
vec_text = token_text.apply(vectorize)

## Vectorization using TF-IDF

In [437]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf = text.apply(tfidf_vectorizer.fit_transform)
tfidf.shape

(10876,)

In [431]:
text.head()

id
1    [our, deed, be, the, Reason, of, this, earthqu...
4        [forest, fire, near, La, Ronge, Sask, Canada]
5    [all, resident, ask, to, shelter, in, place, b...
6    [13,000, people, receive, wildfire, evacuation...
7    [just, got, send, this, photo, from, Ruby, Ala...
Name: text, dtype: object

In [468]:
print(text[0], tfidf[0])

['just', 'happen', 'a', 'terrible', 'car', 'crash']   (0, 3)	1.0
  (1, 2)	1.0
  (3, 4)	1.0
  (4, 0)	1.0
  (5, 1)	1.0


In [469]:
# train_vec = tfidf[0:len(train_data)]
# test_vec = tfidf[len(train_data):]

In [509]:
print(len(train_data), len(train_vec))

7613 7613


## Logistic regression

In [55]:
train_vec = text[0:len(train_data)]
test_vec = text[len(train_data):]

In [56]:
y = train_data.target
X = train_vec

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [58]:
text

id
1               [deed, Reason, earthquake, ALLAH, forgive]
4                [forest, fire, near, Ronge, Sask, Canada]
5        [resident, shelter, place, notify, officer, ev...
6        [13,000, people, receive, wildfire, evacuation...
7        [send, photo, Ruby, Alaska, smoke, wildfires, ...
                               ...                        
10861    [earthquake, SAFETY, ANGELES, SAFETY, fastener...
10865    [storm, hurricane, city&amp;3other, hard, yard...
10868                   [Green, Line, derailment, Chicago]
10874                 [issue, Hazardous, Weather, Outlook]
10875    [CityofCalgary, activate, Municipal, Emergency...
Name: text, Length: 10876, dtype: object

In [62]:
tmpx = text[1:3]

In [66]:
tmpx

id
4                   forest fire near Ronge Sask Canada
5    resident shelter place notify officer evacuati...
Name: text, dtype: object

In [65]:
tmpx[4] = "forest fire near Ronge Sask Canada"
tmpx[5] = "resident shelter place notify officer evacuation shelter place order expect"

In [67]:
tmpy = y[1:3]

In [69]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(solver='lbfgs')
log.fit(tmpx,tmpy)

ValueError: could not convert string to float: 'forest fire near Ronge Sask Canada'

## Train

In [563]:
train_vec = vec_text[0:len(train_data)]
test_vec = vec_text[len(train_data):]

In [527]:
y = train_data.target

In [566]:
X = train_vec

In [567]:
X = np.array(X).reshape((-1,1))
X.shape

(7613, 1)

In [568]:
X[0]

array([list([array([-0.02774072, -0.11559065,  0.14615542, -0.11262162,  0.20590073,
        1.3233712 ,  0.20108943, -0.48430508, -0.42471087,  0.26066253,
        0.38413414,  0.41469753,  0.64132434,  0.04055576, -1.3105894 ,
        1.1406448 ,  0.4976621 ,  0.0936054 , -0.27008253,  0.72645736,
        0.05013415,  0.59617174,  0.13145459,  0.69075036, -0.71736395,
        0.12233558,  0.38195804, -0.01476383,  0.12947941, -0.27279308,
        0.39406896, -1.1618849 , -0.19090304, -1.3933973 ,  0.08491737,
       -0.75081015,  0.94009125,  0.8038806 ,  0.23661709,  0.27275795,
       -0.05207478,  0.13427894, -0.8324771 ,  1.1079587 ,  0.2104932 ,
        0.62744683, -0.2664832 ,  1.0130234 ,  0.81135255, -0.34865883,
        0.24815471,  1.5028428 , -0.91374344, -0.18855765,  0.84990066,
       -1.4090112 , -1.0745279 , -0.12779333,  0.10113759,  0.49894562,
       -0.42008552,  0.48094767, -0.4808635 , -0.5301091 ,  0.49211234,
        0.97112507, -0.20914365,  0.3704738 , -1.68

In [552]:
y = np.array(y).reshape((-1,1))


In [569]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [48]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

/Users/yulialysenko/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


ValueError: Please reshape the input data X into 2-dimensional matrix.

In [12]:
cat_features = ['keyword', 'location']

In [571]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    #loss_function='CrossEntropy'
)


# clf.fit(X_train, y_train, 

# )

In [572]:
clf.fit(X_train, y_train)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="[array([ 0.3495788 , -0.4561724 ,  1.2586519 , -0.7471764 , -0.46494156,
        0.16647986, -0.10573772, -0.8771249 , -0.65399534, -0.15204024,
        0.07033759,  0.46884963, -0.19904914,  0.77769816, -0.8107884 ,
        1.0268824 ,  0.7427417 , -0.9502356 ,  0.39993343, -0.63472307,
        0.18649387,  0.17406642, -0.656852  ,  1.1624869 , -0.87541837,
        0.33441642,  0.7458436 , -1.0898608 , -0.23791698,  1.451618  ,
       -0.33459514, -0.9694805 , -0.44435063, -0.69399947,  1.1743393 ,
        0.5280838 , -1.577163  , -0.28677857, -1.0974559 ,  1.1254141 ,
       -0.4843909 , -0.5178052 , -0.71023196,  0.8638295 , -0.62591344,
        0.30161053,  0.6015882 ,  1.0868223 ,  0.02368483,  0.28823793,
        0.16170247,  0.40672344, -0.05991697,  0.13433132,  0.6788314 ,
       -1.2013595 , -0.01681021,  0.39632112,  1.1827209 ,  0.7607031 ,
       -1.0643141 , -0.34678566, -1.0176688 , -0.8860060